In [1]:
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
base_path = os.path.abspath(os.pardir)
sys.path.append(base_path + '/reconyx_classifier')

os.chdir(base_path + '/data')

from data_utils import io as data_reader

# Data visualization

We want to look at the full data before splitting them. In particular, check of long the average series of event pictures is.

In [2]:
# read metadata of all files to explore
if False:
    data = data_reader.read_training_metadata(base_path + "/data/source_data_v2/", ['unknown', 'cheetah', 'leopard'], extend_events=True, relative_paths=True)
    data.to_hdf(base_path + "/data/source_data_v2/metadata_raw.hdf5", "df")
else:
    data = pd.read_hdf(base_path + "/data/source_data_v2/metadata_raw.hdf5")
    #data2 = pd.read_hdf(base_path + "/data/model_data/metadata.hdf5")

In [3]:
data

,ambient_temp,brightness,contrast,datetime,event1,event2,filename,hour,path,saturation,...,sequence_max,serial_no,sharpness,event_key_simple,sortkey,label,set,duplicates,timeoffset,event_key
0,4,0,160,2015-05-12 05:16:20,0,61,Leo_Omatewa_000408.jpg,5,source_data_v2/leopard/Leo_Omatewa_000408.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2015_132_61,H600HG01173547_2015_132_611431407780000000000,leopard,none,0,NaT,H600HG01173547_2015_132_61
1,4,0,160,2015-05-12 05:16:22,0,61,Leo_Omatewa_000409.jpg,5,source_data_v2/leopard/Leo_Omatewa_000409.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2015_132_61,H600HG01173547_2015_132_611431407782000000000,leopard,none,0,0 days 00:00:02,H600HG01173547_2015_132_61
2,14,0,160,2017-05-08 18:56:30,0,587,Eska03_Leo_B_000020.jpg,18,source_data_v2/leopard/Eska03_Leo_B_000020.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2017_128_587,H600HG01173547_2017_128_5871494269790000000000,leopard,none,1,727 days 13:40:08,H600HG01173547_2017_128_587
3,14,0,160,2017-05-08 18:56:30,0,587,Leopard_000051.jpg,18,source_data_v2/leopard/Leopard_000051.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2017_128_587,H600HG01173547_2017_128_5871494269790000000000,leopard,none,1,0 days 00:00:00,H600HG01173547_2017_128_587
4,14,0,160,2017-05-08 18:56:31,0,587,Eska03_Leo_B_000021.jpg,18,source_data_v2/leopard/Eska03_Leo_B_000021.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2017_128_587,H600HG01173547_2017_128_5871494269791000000000,leopard,none,1,0 days 00:00:01,H600HG01173547_2017_128_587
5,14,0,160,2017-05-08 18:56:31,0,587,Leopard_000052.jpg,18,source_data_v2/leopard/Leopard_000052.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2017_128_587,H600HG01173547_2017_128_5871494269791000000000,leopard,none,1,0 days 00:00:00,H600HG01173547_2017_128_587
7,14,0,160,2017-05-08 18:56:32,0,587,Eska03_Leo_B_000022.jpg,18,source_data_v2/leopard/Eska03_Leo_B_000022.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2017_128_587,H600HG01173547_2017_128_5871494269792000000000,leopard,none,1,0 days 00:00:01,H600HG01173547_2017_128_587
6,14,0,160,2017-05-08 18:56:32,0,587,Leopard_000053.jpg,18,source_data_v2/leopard/Leopard_000053.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2017_128_587,H600HG01173547_2017_128_5871494269792000000000,leopard,none,1,0 days 00:00:00,H600HG01173547_2017_128_587
8,14,0,160,2017-05-08 18:56:34,0,588,Eska03_Leo_B_000023.jpg,18,source_data_v2/leopard/Eska03_Leo_B_000023.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2017_128_588,H600HG01173547_2017_128_5881494269794000000000,leopard,none,1,0 days 00:00:02,H600HG01173547_2017_128_587
9,14,0,160,2017-05-08 18:56:34,0,588,Leopard_000054.jpg,18,source_data_v2/leopard/Leopard_000054.jpg,0,...,3,H600HG01173547,32,H600HG01173547_2017_128_588,H600HG01173547_2017_128_5881494269794000000000,leopard,none,1,0 days 00:00:00,H600HG01173547_2017_128_587


In [ ]:
#print(data.groupby('event_key_simple').size().describe())
# plot distribution of images
plt.subplot(1, 2, 1)
hist_data = plt.hist(data.groupby('event_key_simple').size())
print(hist_data)

# how much time did the series span on average?
plt.subplot(1, 2, 2)
plt.hist((data.groupby('event_key_simple').last()['datetime'].subtract(data.groupby('event_key_simple').first()['datetime'])).dt.seconds)
plt.show()

sum(data.groupby('event_key_simple').last()['datetime'].subtract(data.groupby('event_key_simple').first()['datetime']).dt.seconds == 0)
print(sum(data.duplicates==0))
print(sum(data.duplicates==1))
print(sum(data.duplicates==2))
print(len(data))

Most of the events consist of **3** shots, the rest are some outliers of 1,2 or 6 pictures. However, some of the 6 picture series seem to be duplicates.

The reader function marked duplicates, distinguishing between normal duplicates and duplicates with conflicting labels. If we plot them, the events with more images are gone.

In [ ]:
non_dup_data = data[data.duplicates==0]

plt.subplot(1, 2, 1)
hist_data = plt.hist(non_dup_data.groupby('event_key_simple').size())
print(hist_data)

plt.subplot(1, 2, 2)
plt.hist((non_dup_data.groupby('event_key_simple').last()['datetime'].subtract(non_dup_data.groupby('event_key_simple').first()['datetime'])).dt.seconds)
plt.show()

In [ ]:
# Copy duplicates to separate directories
#import shutil

for group in data[data.duplicates==1].groupby('event_key_simple'):
    group_key = group[1].event_key_simple.unique()[0]
    print(group_key)
    #os.mkdir(group_key)
    for index, row in group[1].iterrows():
        #shutil.copy(row['path'], group_key + '/' + row['filename'])
        print(row['filename'], row['path'])

In [ ]:
# Print duplicate naming prefixes and counts
from collections import Counter
names = Counter()
for index, row in data[data.duplicates==1].iterrows():
    names['_'.join(row['filename'].split('_')[:-1])] += 1
    
for value, count in names.most_common():
    print(value, count)

# Data split

We split the image data into training and validation sets. Later we can extract the necessary metadata from the training set to train the network.  
Need to be careful to maintain series of events while shuffling, i.e. only shuffle whole series of events.

To maintain a balance of images, the default validation fraction of 0.1 could be weighted by image series length. But since most series are 3 images long, we just split the image series.

We could also split on the extended events, to be determined if it makes sense.

In [14]:
import shutil

def split_simple(data, train_dir, val_dir, val_frac):
    # assign files to train/val independently
    # but still no duplicates or mislabeled examples
    was_added = set()
    for index, row in data.iterrows():
        # skip items with conflicting labels
        if row['duplicate'] == 2:
            continue

        # only ever add one item that's duplicated
        if row['duplicate'] == 1:
            # if already added, continue
            if row['event_key_simple'] in was_added:
                continue

            # remember we added it
            was_added.add(row['event_key_simple'])

        is_validation = (random.random() < val_frac)
        target_dir = val_dir if is_validation else train_dir
        target_label = 'val' if is_validation else 'train'

        (src_file, dst_file) = (row['path'], target_dir + '/' + row['filename'])
        os.symlink(src_file, dst_file)
        #print("{} --> {}".format(src_file, dst_file))
        
        
# move the files in the group_data to their output directory
# additionally add a metadata label for the set the files are moved to
def move_file_group(data, group_data, target_dir, target_label, check_added = False): 
    was_added = set()
    for index, row in group_data.iterrows():
        if check_added:
            if row['sequence_idx'] in was_added: continue
            was_added.add(row['sequence_idx'])
        
        # move the file to its directory
        (src_file, dst_file) = (row['path'], target_dir + '/' + row['label'] + '/' + row['label'] + '_' + str(index) + '.jpg')
        #print("{} --> {}".format(os.path.relpath(src_file), os.path.relpath(dst_file)))
        #print("{} --> {}".format(src_file, dst_file))
        shutil.copy(src_file, dst_file)
        #os.symlink(src_file, dst_file)
        
        # mark the set the file is moved to in the metadata
        data.loc[index, 'set'] = target_label
        data.loc[index, 'set_path'] = os.path.relpath(dst_file)
        
        
def split_events(data, train_dir, val_dir, val_frac):
    # first, split non-duplicated simple events
    for group in data[data.duplicates == 0].groupby('event_key_simple'):
        is_validation = (random.random() < val_frac)
        target_dir = val_dir if is_validation else train_dir
        target_label = 'val' if is_validation else 'train'
        move_file_group(data, group[1], target_dir, target_label)

    # split duplicated events with same labels, but only move one set of images
    for group in data[data.duplicates == 1].groupby(['event_key_simple']):
        #dedup_group = group[1].groupby('sequence_idx').last()

        is_validation = (random.random() < val_frac)
        target_dir = val_dir if is_validation else train_dir
        target_label = 'val' if is_validation else 'train'
        move_file_group(data, group[1], target_dir, target_label, check_added=True)
        
        
def split_files(data, output_dir, val_frac, keep_events = True):   
    def make_set_dir(dir_name):
        os.makedirs(dir_name)
        os.makedirs(dir_name + '/cheetah')
        os.makedirs(dir_name + '/leopard')
        os.makedirs(dir_name + '/unknown')
    
    train_dir = output_dir + '/train'
    val_dir = output_dir + '/val'

    # create required directories
    if os.listdir(output_dir) != []:
        print("Error: Make sure the output directory is empty!")
        return

    print(train_dir)
    print(val_dir)

    make_set_dir(train_dir)
    make_set_dir(val_dir)
    
    random.seed(a=42)
    
    if keep_events:
        split_events(data, train_dir, val_dir, val_frac)
    else:
        split_simple(data, train_dir, val_dir, val_frac)
    
    data.to_hdf(output_dir + '/metadata.hdf5', "df")

In [16]:
split_files(data, base_path + '/data/model_data_extended', 0.1)

/media/benjamin/Daten/Data/Gepardenvisionsystem_IZW/data/model_data_extended/train
/media/benjamin/Daten/Data/Gepardenvisionsystem_IZW/data/model_data_extended/val


/home/benjamin/Software/miniconda3/envs/ml_env/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['filename', 'path', 'serial_no', 'event_key_simple', 'sortkey', 'label', 'set', 'event_key', 'set_path']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [17]:
print("Training: {}\t Validation: {}\t None: {}".format(sum(data.set=='train'), sum(data.set=='val'), sum(data.set=='none')))

Training: 43118	 Validation: 4729	 None: 1937
